In [0]:
filepathrawlocation = "abfss://raw-data@kaggleseries.dfs.core.windows.net"

In [0]:
df = spark.read.format("csv").option("inferSchema", True).option("header", True).load(filepathrawlocation)

In [0]:
display(df)

In [0]:
print(df.columns)
col_name=df.columns

In [0]:
df = df.toDF(*[col_name.replace('\xa0', ' ').replace('(', '').replace(')','')     for col_name in df.columns])
print(df.columns)

In [0]:
df = df.toDF(*[colName.replace(' ','_').replace('.', '') for colName in df.columns])
print(df.columns)

In [0]:
df.display()

In [0]:
df.drop('Peak', 'All_Time_Peak','Ref').display()

In [0]:
columns_to_fix = ["actual_gross", "tour_title"]

for col_name in columns_to_fix:
    df = df.withColumn(col_name, regexp_replace(col(col_name), r"\[.*?\]|†|\*|\+|‡|â€œ|â€", ""))

df.display()

In [0]:
# from pyspark.sql.functions import regexp_replace, col
# columns_to_fix = ["actual_gross", "tour_title"]
# for col_name in columns_to_fix:
#     df = df.withColumn(
#         col_name,
#         regexp_replace(
#             col(col_name),
#             r"\\[.*?\\]|\\*|\\+|\u2021|\u2020|\u2013|\u201C|\u201D",  # Unicode escapes
#             ""
#         )
#     )
# df.display()


In [0]:
from pyspark.sql.functions import regexp_replace, col

columns_to_fix = ["actual_gross", "tour_title"]

for col_name in columns_to_fix:
    dff = df.withColumn(
        col_name,
        regexp_replace(
            col(col_name),
            r"[^a-zA-Z0-9 ]",  # keep only letters, numbers, spaces
            ""
        )
    )

dff.display()


In [0]:
df.display()

In [0]:
for colname in ['actual_gross', 'Adjusted_gross_in_2022_dollars', 'Average_gross']:
    df=df.withColumn(colname, regexp_replace(col(colname), r"[$,]", ""))


df.display()

In [0]:
df = df.withColumn("actual_gross", col("actual_gross").cast("double"))
df = df.withColumn("Adjusted_gross_in_2022_dollars", col("Adjusted_gross_in_2022_dollars").cast("double"))

df = df.withColumn("Average_gross", col("Average_gross").cast("double"))


In [0]:
df.display()

In [0]:
df.printSchema()

In [0]:
from pyspark.sql.functions import split, col, expr

df = (
    df.withColumn("start_year", expr("cast(get(split(Years, '–'), 0) as int)"))
      .withColumn("end_year", expr("cast(get(split(Years, '–'), 1) as int)"))
)


In [0]:
df.display()

In [0]:
df.columns

In [0]:
df = df.drop('Peak', 'All_Time_Peak', 'Ref')

In [0]:
desiredOrder = [
    'Rank',
    'Artist',
    'Shows',
    'tour_title',
    'Years',
    'start_year',
    'end_year',
    'actual_gross',
    'Adjusted_gross_in_2022_dollars',
    'Average_gross'
]

df = df.select(*desiredOrder)
display(df)

In [0]:
df = df.withColumnRenamed("Adjusted_gross_in_2022_dollars", "Adjusted_gross")

In [0]:
df.display()

In [0]:
outputpath = "abfss://transformed-data@kaggleseries.dfs.core.windows.net/"

In [0]:
df.write.mode("overwrite").parquet(outputpath)